---
**Creating a dataset**

In [1]:
import spacy
import csv
import pandas as pd
import numpy as np
import re
from collections import defaultdict
from pymorphy3 import MorphAnalyzer
import warnings
warnings.filterwarnings("ignore")
nlp = spacy.load("ru_core_news_lg")
all_stopwords = nlp.Defaults.stop_words

In [2]:
df = pd.read_csv('lenta-ru-news.csv')

In [3]:
df.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [4]:
texts = df['text'][:100000]

In [7]:
texts = texts.dropna()

In [8]:
morph = MorphAnalyzer()
punctuation_list = ['.', ',', ':', '?', '!', ';', '(', ')', '[', ']', '{', '}', '-', '―']
pattern = r'[A-Za-z0-9!#$%&\'()*+«»,…./:;<=>№?@[\]^_`{|}~—\"\-]+'

In [9]:
def toProcess(text):
    text_edited = re.sub(pattern, ' ', text).strip()
    
    #  Separating the text on the sentences
    text_splitted = text_edited.split('.')

    # Tokenization
    wordbase = list()
    for sentence in text_splitted:
        tokens = sentence.strip().split()
        
        #  Lemmatization
        for token in tokens:
            token_lemmatized = morph.normal_forms(token)[0]
            
            if token_lemmatized not in wordbase and token_lemmatized not in all_stopwords:
                if len(token_lemmatized) >= 3:
                    wordbase.append(token_lemmatized)
    return wordbase

In [10]:
result = texts.apply(toProcess)

In [11]:
def export_format(text):
    return ' '.join(text)

export = result.apply(export_format)
export.to_csv('words.csv')

---
**Top words**

In [12]:
words_dict = defaultdict(int)
for token_set in result:
    for token in token_set:
        words_dict[token] += 1

In [13]:
len(words_dict)

148724

In [14]:
sorted(words_dict, key=words_dict.get, reverse=True)[:5]

['сообщать', 'год', 'время', 'заявить', 'сообщить']

---
**Training a model Word2Vec**

In [2]:
from gensim.models import Word2Vec

In [3]:
words_df = pd.read_csv('words.csv', usecols=['text'])
words_df.head()

,text
0,бой сопоцкина друскеник закончиться отступлени...
1,министерство народный просвещение вид происход...
2,штабс капитан нестеров день увидеть район желт...
3,фотограф корреспондент рассказывать случай пор...
4,лицо приехать варшава люблин передавать достав...


In [4]:
print(len(words_df))
words_df = words_df[words_df.text.str.len() > 2]
print(len(words_df))

100000
99895


In [5]:
def import_format(text):
    return text.split()
words_df = words_df['text'].apply(import_format)
words_df.head()

0    [бой, сопоцкина, друскеник, закончиться, отсту...
1    [министерство, народный, просвещение, вид, про...
2    [штабс, капитан, нестеров, день, увидеть, райо...
3    [фотограф, корреспондент, рассказывать, случай...
4    [лицо, приехать, варшава, люблин, передавать, ...
Name: text, dtype: object

In [6]:
model = Word2Vec(
    min_count=10,
    window=2,
    vector_size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

In [7]:
model.build_vocab(words_df)

In [8]:
model.train(words_df, total_examples=model.corpus_count, epochs=30, report_delay=1)

(153440060, 240673680)

In [30]:
model.wv.most_similar(positive=["министерство"])

[('министр', 0.6184762716293335),
 ('ведомство', 0.5176082849502563),
 ('департамент', 0.5128709673881531),
 ('оборона', 0.4982307255268097),
 ('замминистра', 0.49637529253959656),
 ('правительство', 0.4938677251338959),
 ('абсорбция', 0.4652884900569916),
 ('юстиция', 0.44359302520751953),
 ('управление', 0.44205033779144287),
 ('минобороны', 0.43186822533607483)]

In [10]:
model.wv.vectors.shape

(27867, 300)

In [11]:
# model.init_sims(replace=True)
model.save('w2v.model')

---
**Loading the model Word2Vec**

In [12]:
model_loaded = Word2Vec.load('w2v.model')

In [13]:
model_loaded.build_vocab(words_df, update=True)

In [25]:
model_loaded.train(words_df, total_examples=model_loaded.corpus_count, epochs=30, report_delay=1)

(2005820, 5072880)

In [15]:
model_loaded.wv.most_similar(positive=["министерство"])

[('министр', 0.6184762716293335),
 ('ведомство', 0.5176082849502563),
 ('департамент', 0.5128709673881531),
 ('оборона', 0.4982307255268097),
 ('замминистра', 0.49637529253959656),
 ('правительство', 0.4938677251338959),
 ('абсорбция', 0.4652884900569916),
 ('юстиция', 0.44359302520751953),
 ('управление', 0.44205033779144287),
 ('минобороны', 0.43186822533607483)]

In [14]:
model_loaded.wv.vectors.shape

(27867, 300)

In [ ]:
model_loaded.save('w2v.model')

---